### Проверка того, что наше API  CardioSpike работает

Сервис **CardioSpike** по предсказанию COVID-19 аномалий в ритме сердца установлен на стандартный хостинг в
режиме Virtual Private Server (https://en.wikipedia.org/wiki/Virtual_private_server).
В данной конфигурации архитектура не подразумевает разделения на компоненты Front и API Backend.
Однако допустимо например запустить 2 копии сервера, чтобы 1 сервер был в роли Front,
а второй в роли API Backend.

- Адрес сервера: server5.hosting.reg.ru
- IP адрес: 31.31.198.106
- Основной тестируемый API http://localhost:5000/predict [POST]

Формат передаваемых данных:
``{'id': [ id0, id1, ... ], 'x': [x0, x1, ...]}``, где
- id - идентификатор пациента
- x - измерения пульса R-R

Преобразования данных:
- Сериализация в строку JSON - ``json.dumps``
- Кодирование в utf-8 - ``str.encode('utf-8')``

In [42]:
SERVER_ENDPOINT = 'http://localhost:5000/predict'

In [43]:
import pandas as pd
from sklearn.metrics import f1_score
from urllib import request

Загрузка данных для тестирвоания

In [44]:
df = pd.read_csv('data/train.csv')
X_test = df[['id', 'x']]
y_test = df['y']
X_test.head()

,id,x
0,1,800
1,1,780
2,1,792
3,1,820
4,1,804


In [45]:
import urllib.request
import json


def get_prediction(id_, x):
    body = {'id': id_, 'x': x}
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')  # needs to be bytes

    req = urllib.request.Request(SERVER_ENDPOINT)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    req.add_header('Content-Length', len(jsondataasbytes))

    response = urllib.request.urlopen(req, jsondataasbytes)

    data = json.loads(response.read())

    return (data['predictions']), (data['diagnosis']), (data['pattern_per_5minute'])

Подготовка данных. Выборка для пациента id=3

In [46]:
X_test_1 = X_test.loc[X_test['id'] == 3]
y_test_1 = y_test.loc[X_test_1.index]
id_ = list(X_test_1['id'])
x = list(X_test_1['x'])

Вызов внешнего API

In [47]:
pred, diagnosis, pattern_per_5minute = get_prediction(id_, x)

Результат

In [48]:
print(f'Пациент \t id={list(diagnosis.keys())[0]}')
print(f'Диагноз: \t {"положительный" if list(diagnosis.values())[0] else "отрицательный"}')
print(f'Показатель: \t {list(pattern_per_5minute.values())[0]} А/мин')
print(f'Количество аномалий: \t {sum(pred)}')
print('--------------------------------------')
print(f'Точность классификации (f1_score): {f1_score(y_test_1, pred)}')

Пациент 	 id=3
Диагноз: 	 положительный
Показатель: 	 11 А/мин
Количество аномалий: 	 30
--------------------------------------
Точность классификации (f1_score): 0.8
